In [1]:
import time

In [2]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Reshape, Conv2DTranspose, Conv2D, Flatten, Dropout, LeakyReLU
from tensorflow.keras.optimizers import Adam
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image

%matplotlib inline

In [3]:
def generator():
    model = Sequential()
    model.add(Dense(20*20*64, activation='relu', input_dim=64))
    model.add(Reshape((20,20,64)))#20
    model.add(Conv2DTranspose(64, (4,4), strides=(3,3), padding='same', activation='relu'))#60
    model.add(Conv2DTranspose(64, (4,4), strides=(2,2), padding='same', activation='relu'))#60
    model.add(Conv2D(3, (5,5), activation='sigmoid', padding='same'))
    return model

In [4]:
def discriminator():
    '''return discriminator model'''
    model = Sequential()
    model.add(Conv2D(32, (4,4), strides=(2,2), activation='relu', padding='same', input_shape=(120,120,3)))
    model.add(Dropout(0.8))
    model.add(Conv2D(32, (4,4), strides=(3,3), activation='relu', padding='same'))
    model.add(Dropout(0.5))
    model.add(Flatten())
    model.add(Dense(1, activation='sigmoid'))
    model.compile(loss='binary_crossentropy',optimizer=Adam())
    return model

In [5]:
discriminator().summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 60, 60, 32)        1568      
_________________________________________________________________
dropout (Dropout)            (None, 60, 60, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 20, 20, 32)        16416     
_________________________________________________________________
dropout_1 (Dropout)          (None, 20, 20, 32)        0         
_________________________________________________________________
flatten (Flatten)            (None, 12800)             0         
_________________________________________________________________
dense (Dense)                (None, 1)                 12801     
Total params: 30,785
Trainable params: 30,785
Non-trainable params: 0
____________________________________________________

In [6]:
generator().summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 25600)             1664000   
_________________________________________________________________
reshape (Reshape)            (None, 20, 20, 64)        0         
_________________________________________________________________
conv2d_transpose (Conv2DTran (None, 60, 60, 64)        65600     
_________________________________________________________________
conv2d_transpose_1 (Conv2DTr (None, 120, 120, 64)      65600     
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 120, 120, 3)       4803      
Total params: 1,800,003
Trainable params: 1,800,003
Non-trainable params: 0
_________________________________________________________________


In [7]:
def generate_latent_vector(n_samples):
    '''generate vector in latent dimension'''
    return np.random.rand(n_samples,64)

In [8]:
def generate_fake_samples(generator, n_samples):
    '''generate fake samples from random numbers'''
    fake_input = generate_latent_vector(n_samples)
    return generator.predict(fake_input), np.zeros((n_samples, 1))

In [9]:
def generate_real_samples(dataset, n_samples):
    mask = np.random.choice(range(dataset.shape[0]),n_samples)
    return dataset[mask,:,:,:], np.ones((n_samples, 1))

In [10]:
def gan(g, d):
    discriminator.trainable = False
    model = Sequential()
    model.add(g)
    model.add(d)
    model.compile(loss='binary_crossentropy',optimizer=Adam())
    return model

In [11]:
def train(g, d, gan, dataset, n_epochs=100, n_batch=256):
    batch_per_epoch = int(dataset.shape[0]/n_batch)
    for i in range(n_epochs):
        for j in range(batch_per_epoch):
            # get real
            x_real, y_real = generate_real_samples(dataset, int(n_batch/2))
            # generate fake
            x_fake, y_fake = generate_fake_samples(g, int(n_batch/2))

            # create training
            x, y = np.vstack((x_real, x_fake)), np.vstack((y_real, y_fake))

            # update discriminator
            d_loss = d.train_on_batch(x,y)

            # generate fake
            x_gan = generate_latent_vector(n_batch)
            y_gan = np.ones((n_batch,1))

            # train generator
            gan_loss = gan.train_on_batch(x_gan, y_gan)

            # summarize
            print('>Epoch: %d, %d/%d, d loss=%.3f, gan loss=%.3f' % (i+1, j+1, batch_per_epoch, d_loss, gan_loss))

In [12]:
g = generator()
d = discriminator()
gan = gan(g, d)

In [13]:
dataset = np.load('../data/fusions.npy')

In [14]:
dataset.shape

(22349, 120, 120, 3)

In [ ]:
t0 = time.time()
train(g, d, gan, dataset, n_epochs=1, n_batch=256)
time.time() - t0